In [26]:
import pandas as pd
import sys,random

In [ ]:
df=pd.read_excel('题目/附件1：数据集1-终稿.xlsx', engine='openpyxl',skiprows=[0])

In [111]:
df

,编号,X坐标（单位: m）,Y坐标（单位:m）,Z坐标（单位: m）,校正点类型,第三问点标记
0,0,0.000000,50000.000000,5000.000000,A 点,0
1,1,33070.825831,2789.480761,5163.524680,0,0
2,2,54832.887019,49179.219108,1448.303791,1,1
3,3,77991.545911,63982.175286,5945.823038,0,0
4,4,16937.179535,84714.341903,5360.293002,0,0
...,...,...,...,...,...,...
608,608,45789.201490,21191.207906,440.001872,1,0
609,609,94917.734859,82958.725321,6169.657707,0,0
610,610,14870.601682,95939.173362,8248.836185,0,0
611,611,93009.567446,4549.333260,7882.608297,1,0


In [112]:
df=df.drop(df.columns[0],axis=1)

In [113]:
point_list= df.values

In [114]:
point_list

array([[0.0, 50000.0, 5000.0, 'A 点', 0],
       [33070.825830821, 2789.48076085272, 5163.5246804925, 0, 0],
       [54832.8870194109, 49179.2191080384, 1448.30379093285, 1, 1],
       ...,
       [14870.6016822319, 95939.1733623643, 8248.83618468285, 0, 0],
       [93009.5674461602, 4549.3332597056, 7882.60829650082, 1, 0],
       [100000.0, 59652.3433795158, 5022.00116448164, 'B点', 0]],
      dtype=object)

In [115]:
point_num = len(point_list)

In [116]:
point_num

613

In [117]:

def get_distance(start_index, end_index):
    x1, y1, z1 = point_list[start_index][0:3]
    x2, y2, z2 = point_list[end_index][0:3]
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2) ** 0.5


def judge_z(start_index, end_index=point_num - 1):
    x1, y1, z1 = point_list[start_index][0:3]
    x2, y2, z2 = point_list[end_index][0:3]
    if abs(z1 - z2) > 5000:
        return False
    else:
        return True

def judge(start_index, end_index, horizontal_error, vertical_error):
    end_point_type = point_list[end_index][3]
    distance = get_distance(start_index, end_index)

    delta_error = distance * sigma
    end_point_horizontal_error = horizontal_error + delta_error
    end_point_vertical_error = vertical_error + delta_error
    if judge_z(start_index):
        if end_index != point_num - 1:  # ????????
            if end_point_type == 0:  # ??
                if end_point_horizontal_error <= beta2 and end_point_vertical_error <= beta1:
                    is_pass = True
                else:
                    is_pass = False
            elif end_point_type == 1:  # ??
                if end_point_horizontal_error <= alpha2 and end_point_vertical_error <= alpha1:
                    is_pass = True
                else:
                    is_pass = False
            else:
                is_pass = False
        else:  # ???????
            if end_point_horizontal_error <= theta and end_point_vertical_error <= theta:
                is_pass = True
            else:
                is_pass = False
    else:
        is_pass = False
    after_end_point_horizontal_error = end_point_horizontal_error
    after_end_point_vertical_error = end_point_vertical_error
    if is_pass:
        if end_point_type == 0:
            after_end_point_horizontal_error = 0
            after_end_point_vertical_error = end_point_vertical_error
        elif end_point_type == 1:
            after_end_point_horizontal_error = end_point_horizontal_error
            after_end_point_vertical_error = 0
    return is_pass, end_point_horizontal_error, end_point_vertical_error,after_end_point_horizontal_error, after_end_point_vertical_error


def rank_distance(point_index_list):
    ranked_list = sorted(point_index_list, key=lambda index_list: get_distance(index_list[0], point_num - 1))
    return ranked_list


def get_all_distance(index_list):
    distance = 0
    for i in range(len(index_list) - 1):
        start_index = index_list[i]
        end_index = index_list[i + 1]
        distance = distance + get_distance(start_index, end_index)
    return distance

In [118]:
def find_path(start_index=0, horizontal_error=0, vertical_error=0):
    global temp_all_distance
    global order, temp_order
    order.append(start_index)
    candidate_list = []
    for index in range(point_num):
        if index != start_index:
            is_pass, end_point_horizontal_error, end_point_vertical_error, after_end_point_horizontal_error, after_end_point_vertical_error = judge(start_index, index, horizontal_error, vertical_error)
            if is_pass and get_distance(start_index, point_num - 1) > get_distance(index,
                                                                       point_num - 1):
                candidate_list.append(index)
    if len(candidate_list) == 0:
        order.pop()
        return
    candidate_list.sort(key=lambda index_list: get_distance(index_list, point_num - 1))
# random.shuffle(candidate_list)
    if len(candidate_list) >= 5:
        candidate_list = candidate_list[0:5]
    for candidate in candidate_list:
        if candidate == point_num - 1:
            order.append(candidate)
            all_distance = get_all_distance(order)
            if all_distance < temp_all_distance:
                temp_all_distance = all_distance
                temp_order = order.copy()
                print('\n' + 'small', order, all_distance)
            order.pop()
            break
        if len(order) > 13:
            break
        is_pass, end_point_horizontal_error, end_point_vertical_error, after_end_point_horizontal_error, after_end_point_vertical_error = judge(start_index, candidate, horizontal_error, vertical_error)
        find_path(candidate, after_end_point_horizontal_error, after_end_point_vertical_error)

    order.pop()
    return

In [119]:

random.seed(1)
sys.setrecursionlimit(100000)
alpha1 = 25
alpha2 = 15
beta1 = 20
beta2 = 25
theta = 30
sigma = 0.001

In [120]:
order = []
temp_all_distance = 1000000
temp_order = []

In [ ]:
find_path(0,0,0)


small [0, 303, 199, 15, 418, 11, 450, 3, 397, 612] 112329.74603335514

small [0, 303, 199, 15, 418, 11, 450, 286, 368, 612] 111362.31025478707

small [0, 303, 199, 15, 418, 11, 450, 286, 397, 612] 111046.2654120447

small [0, 303, 199, 15, 418, 11, 369, 214, 397, 612] 110668.61320892324

small [0, 303, 199, 15, 418, 278, 369, 214, 397, 612] 107253.70811782636

small [0, 303, 199, 15, 56, 278, 369, 214, 397, 612] 106879.27501719673

small [0, 303, 199, 15, 148, 278, 369, 214, 397, 612] 105865.20026694027


In [92]:
temp_order

[0, 455, 338, 275, 610]